In [28]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from crops_package.data import split_and_reduce_data
from tensorflow.keras.callbacks import EarlyStopping


In [20]:
data = pd.read_csv(f"{os.environ.get('DATA_PATH')}data.csv")
data['filename'] = os.environ.get("LOCAL_PATH") + data['filename']
data['class'] = data['class'].astype(str)

# train_df, val_df, test_df = split_data(data)
train_df, val_df, test_df = split_and_reduce_data(data, 0.90)

from crops_package.data import preprocessor_df
train_generator = preprocessor_df(train_df)
val_generator = preprocessor_df(val_df)
test_generator = preprocessor_df(test_df)

# from tensorflow.keras import mixed_precision
# mixed_precision.set_global_policy(policy="mixed_float16")
# mixed_precision.global_policy()

Found 1818 validated image filenames belonging to 22 classes.
Found 321 validated image filenames belonging to 22 classes.
Found 378 validated image filenames belonging to 22 classes.


Found 378 validated image filenames belonging to 22 classes.


In [21]:
pretrained_model = tf.keras.applications.ResNet101V2(
        include_top=False ,
        input_shape=[128, 128, 3]
    )
pretrained_model.trainable = False

model_8_with_callback = tf.keras.Sequential([
        pretrained_model,

        tf.keras.layers.Conv2D(32, (4,4), activation='relu', padding='same'),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),

        tf.keras.layers.Conv2D(32, (4,4), activation='relu', padding='same'),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),

        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(22, activation='softmax')
    ])

### Model compilation
model_8_with_callback.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy', tf.keras.metrics.Precision()],
                run_eagerly=True)

In [9]:
# model_8_with_callback = create_model()

# Loads the weights
model_8_with_callback.load_weights("/home/wouter/code/MahautHDL/save_the_crops/callbacks/callback75acc_80prec/events.out.tfevents.1710184627.59a361c47c3f.34.0.v2".format(epoch=0))

# Re-evaluate the model
loss, acc = model_8_with_callback.evaluate(test_generator, verbose=2)

2024-03-12 09:03:03.313211: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open /home/wouter/code/MahautHDL/save_the_crops/callbacks/callback75acc_80prec/events.out.tfevents.1710184627.59a361c47c3f.34.0.v2: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


OSError: Unable to synchronously open file (file signature not found)

In [34]:
es = EarlyStopping(patience = 5, verbose = 2, restore_best_weights=True)

model_7_pre_train_hist = model.fit(train_generator,
                                   epochs=100,
                                   verbose=1,
                                    callbacks=[es, create_tensorboard_callback("/kaggle/working", # save experiment logs here
                                                                         experiment_name="model_7_pre_train")]
)

ValueError: `y` argument is not supported when using `keras.utils.Sequence` as input.

In [22]:
pretrained_model = tf.keras.applications.ResNet101V2(
        include_top=False ,
        input_shape=[128, 128, 3]
    )
pretrained_model.trainable = False

model_9_with_dense = tf.keras.Sequential([
        pretrained_model,

        tf.keras.layers.Dense(60,activation="relu"),
        tf.keras.layers.Dense(50,activation="relu"),
        tf.keras.layers.Dense(40,activation="relu"),
        tf.keras.layers.Dense(32,activation="relu"),

        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(22, activation='softmax')
    ])

### Model compilation
model_9_with_dense.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy', tf.keras.metrics.Precision()],
                run_eagerly=True)

In [23]:
from crops_package.model import create_tensorboard_callback

es = EarlyStopping(patience = 5, verbose = 2, restore_best_weights=True)

model_9_with_dense_hist = model_9_with_dense.fit(train_generator,
                                   epochs=100,
                                   verbose=1,
                                    callbacks=[es, create_tensorboard_callback(dir_name=os.environ.get("CALLBACK_PATH"), # save experiment logs here
                                                                         experiment_name="model_7_pre_train")]
)

Saving TensorBoard log files to: /home/wouter/code/MahautHDL/save_the_crops/callbacks/model_7_pre_train-20240312-093840
Epoch 1/100
18/57 [========>.....................] - ETA: 3:35 - loss: 3.0448 - accuracy: 0.0660 - precision_4: 0.0000e+00

2024-03-12 09:40:39.233401: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: OSError: broken data stream when reading image file
Traceback (most recent call last):

  File "/home/wouter/.pyenv/versions/3.10.6/envs/save_the_crops/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/home/wouter/.pyenv/versions/3.10.6/envs/save_the_crops/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/home/wouter/.pyenv/versions/3.10.6/envs/save_the_crops/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/wouter/.pyenv/versions/3.10.6/envs/save_the_crops/lib/python3.10/site-packages/keras/src/engine/data_adapter.py", line 917, in wrapped_generator
    for data in generator_fn():

  File "/home/wouter/.pyenv/versio

20/57 [=========>....................] - ETA: 3:23 - loss: 3.0408 - accuracy: 0.0719 - precision_4: 0.0000e+00

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OSError: broken data stream when reading image file
Traceback (most recent call last):

  File "/home/wouter/.pyenv/versions/3.10.6/envs/save_the_crops/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/home/wouter/.pyenv/versions/3.10.6/envs/save_the_crops/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/home/wouter/.pyenv/versions/3.10.6/envs/save_the_crops/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/home/wouter/.pyenv/versions/3.10.6/envs/save_the_crops/lib/python3.10/site-packages/keras/src/engine/data_adapter.py", line 917, in wrapped_generator
    for data in generator

# Model initialization

In [15]:
from tensorflow.keras import layers, regularizers, Sequential
from tensorflow.keras import models

def initialize_model():
    # Create base model
    input_shape = (224, 224, 3)
    base_model = tf.keras.applications.resnet_v2.ResNet101V2(include_top=False)
    base_model.trainable = False # freeze base model layers

    # Create Functional model
    inputs = layers.Input(shape=input_shape, name="input_layer")
    # Note: EfficientNetBX models have rescaling built-in but if your model didn't you could have a layer like below
    # x = layers.Rescaling(1./255)(x)
    x = base_model(inputs, training=False) # set base_model to inference mode only
    x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
    x = layers.Dense(22)(x) # want one output neuron per class
    # Separate activation of output layer so we can output float32 activations
    outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
    model = tf.keras.Model(inputs, outputs)

    # Compile the model
    model.compile(loss="sparse_categorical_crossentropy", # Use sparse_categorical_crossentropy when labels are *not* one-hot
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

    return model

In [5]:
model_7_pre_train = initialize_model()
model_7_pre_train.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 resnet101v2 (Functional)    (None, None, None, 2048   42626560  
                             )                                   
                                                                 
 pooling_layer (GlobalAvera  (None, 2048)              0         
 gePooling2D)                                                    
                                                                 
 dense (Dense)               (None, 22)                45078     
                                                                 
 softmax_float32 (Activatio  (None, 22)                0         
 n)                                                              
                                                             

# fit model with earlystopping

In [23]:
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [46]:
data


,filename,class
0,/home/wouter/code/MahautHDL/save_the_crops/raw...,Cassava brown spot
1,/home/wouter/code/MahautHDL/save_the_crops/raw...,Cassava brown spot
2,/home/wouter/code/MahautHDL/save_the_crops/raw...,Cassava brown spot
3,/home/wouter/code/MahautHDL/save_the_crops/raw...,Cassava brown spot
4,/home/wouter/code/MahautHDL/save_the_crops/raw...,Cassava brown spot
...,...,...
25165,/home/wouter/code/MahautHDL/save_the_crops/raw...,Maize healthy
25166,/home/wouter/code/MahautHDL/save_the_crops/raw...,Maize healthy
25167,/home/wouter/code/MahautHDL/save_the_crops/raw...,Maize healthy
25168,/home/wouter/code/MahautHDL/save_the_crops/raw...,Maize healthy


In [53]:
def preprocess_img(image, label, img_shape=224):
    """
    Converts image datatype from 'uint8' -> 'float32' and reshapes image to
    [img_shape, img_shape, color_channels]
    """
    image = tf.image.resize(image, [img_shape, img_shape]) # reshape to img_shape
    return tf.cast(image, tf.float32), label # return (float32_image, label) tuple

# Map preprocessing function to training data (and paralellize)
train_data = train_df.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map prepreprocessing function to test data
test_data = test_df.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Turn test data into batches (don't need to shuffle)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

TypeError: preprocess_img() missing 1 required positional argument: 'label'

In [18]:
from tensorflow.keras.callbacks import EarlyStopping
from crops_package.model import fit_and_save_model
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

%load_ext tensorboard
from tensorboard import notebook

model_7_pre_train = initialize_model()

model_name = "model_7_pre_train"

es = EarlyStopping(patience = 5, verbose = 2, restore_best_weights=True)

model_7_pre_train_hist = model_7_pre_train.fit(train_df['filename'],
                                               train_df['class'],
                                               callbacks=[es, create_tensorboard_callback(dir_name=os.environ.get("CALLBACK_PATH"), # save experiment logs here
                                                            experiment_name=model_name)])

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Saving TensorBoard log files to: /home/wouter/code/MahautHDL/save_the_crops/callbacks/model_7_pre_train-20240311-153802


ValueError: in user code:

    File "/home/wouter/.pyenv/versions/3.10.6/envs/save_the_crops/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/home/wouter/.pyenv/versions/3.10.6/envs/save_the_crops/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/wouter/.pyenv/versions/3.10.6/envs/save_the_crops/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/home/wouter/.pyenv/versions/3.10.6/envs/save_the_crops/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/home/wouter/.pyenv/versions/3.10.6/envs/save_the_crops/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handl

In [34]:
%tensorboard --logdir ../callbacks
notebook.list() # View open TensorBoard instances


INFO:tensorflow:Assets written to: /Users/thomasverpoort/code/MahautHDL/save_the_crops/models/first_model_07_03/assets


INFO:tensorflow:Assets written to: /Users/thomasverpoort/code/MahautHDL/save_the_crops/models/first_model_07_03/assets


In [31]:
from crops_package.data import check_for_errors_and_create_excel_2

check_for_errors_and_create_excel_2()

Tomato leaf blight finished.
Tomato leaf curl finished.
Tomato verticulium wilt finished.
Tomato healthy finished.
Tomato septoria leaf spot finished.
Tomato csv has been saved.
Cassava brown spot finished.
Cassava mosaic finished.
Cassava green mite finished.
Cassava healthy finished.
Cassava bacterial blight finished.
Cassava csv has been saved.
Cashew leaf miner finished.
Cashew gumosis finished.
Cashew red rust finished.
Cashew healthy finished.
Cashew anthracnose finished.
Cashew csv has been saved.
Maize leaf blight finished.
Maize fall armyworm finished.
Maize grasshoper finished.
Maize streak virus finished.
Maize leaf beetle finished.
Maize leaf spot finished.
Maize healthy finished.
Maize csv has been saved.
all saved
